tracking_avi_movies_and_saving_movie_of_trajectories.ipynb
This code is for tracking one or two movies and comparing the results.
Each movie is assumed to have red and green particles.

# First time? Install everything here!

In [ ]:
"""### Only run this code if necessary to install packages 
### on a new anaconda installation. 
### (Remove the quotes before and after to uncomment.)

!conda install conda --yes
!conda update conda --yes
!conda install -c conda-forge trackpy ffmpeg tifffile av --yes
#!conda update -c conda-forge trackpy ffmpeg tifffile av --yes
%pip install pims moviepy opencv-python numba seaborn scipy -U
# !pip3 install opencv-python  
# https://stackoverflow.com/questions/46610689/how-to-import-cv2-in-python3"""

# Packages

In [ ]:
try:
    import matplotlib as mpl
    import matplotlib.pyplot as plt

    import numpy as np
    import pandas as pd

    import pims
    import gc
    import trackpy as tp
    import os
    import time
    import numba
    import tifffile
    import imagecodecs
    import scipy.optimize as sco
    import cv2
    import scipy
    import random

    import math
    import seaborn

    import datetime
    import time

    from numpy import sqrt
except:
    print("You need to run the installation code. It's commented out with triple quotation marks in the above cell.")
    raise

try:
    from rotational_drift_subtraction import drift_subtract 
except:
    print("You need to put the file rotational_drift_subtraction.py in the same folder as this ipynb file", 
          "or add the appropriate folder to your python path.")
    raise

%matplotlib inline

# Read these instructions!

In [ ]:
# Setup Instructions: 
# All the lines in this cell need to be set in order to identify the movie data.

# You want a folder inside the Data folder whose name is the date the data was taken in the
# format yyyy-mm-dd. Set data_date below to that date. 

# Uncomment the proper scaling amount, and
# set the appropriate variable for whether you're analyzing fueled and/or control data.

# movies should be already saved to the [home]Data\[2021-10-20]\
# or you can replace the movie_data_folder line below with wherever your movies are located.
# Note: on Windows you must use the letter r before quoted disk locations 
# so that it properly interprets the backslash.

## Are you using tif files (a series of files for one movie) or is the whole movie one file?
# If you just have one file for each movie, 
# then you can use control_movie_filename and fueled_movie_filename

# If you have multiple files for each movie (common for tif files):
# Look at your data folder. If you have tif movies that you don't want to analyze but 
# which start with the same start as your movies that you do want to analyze, 
# move the other movies to a "not_to_analyze" folder or similar.
# The notebook will open every tif file in that
# folder that starts with fueled_movie_start and/or control_movie_start.
# Set movie_name below to the portion of the name that they share. 

# If you are using blue particles, then search for ****** and switch from grayscale to bluegreen.

# Once you have set this cell, 
# then this entire notebook
# is ready to run!

gc.enable() # garbage collector

today = datetime.date.today().isoformat()  # automatically knows the analysis date is today.
data_date = '2023-04-26'             # what date was the data taken?
home = r'C:\Users\vhorowit\Desktop\BessLawrence'     # what is the user's folder that has Data and Analysis folders
movie_name = '200 & 20000 PEG @ 25mg per mL (1)'     # some identifier for this set of movies.
doing_fueled = False                        # do you want to analyze a fueled movie?
doing_control = True                      # do you want to analyze a control (unfueled) movie?
using_tif = False                           # is your movie a tif file or something else?
do_slow_stuff = True                        # show trajectories
colorseparation = False                      # colorseparation True: red and green particles -- track them separately
                                             # colorseparation False: analyze one color of particles

assert (doing_fueled or doing_control) == True, "Must have something to analyze!"

# scaling, measured in microns per pixel
#scaling = 330 / 1247.96 # thorcam 20x1.0, measured 2021-06-17
#scaling = 220 / 1250.04 # thorcam 20x1.5, measured 2021-06-17
#scaling = 150 / 1127.54 # thorcam 40x1.0, measured 2021-06-16
#scaling = 100 / 1130.61 # thorcam 40x1.5, measured 2021-06-16
#scaling = 80 / 914.92 # thorcam 60x1.0, measured 2021-05-28
#scaling = 60 / 1031.07 # thorcam 60x1.5, measured 2021-05-28

### movies should be already saved to the folder [home]Data\[data_date]\
### or edit as needed to indicate where the movies are stored.
data_folder = os.path.join(home, 'Data')
movie_data_folder = os.path.join(data_folder, data_date)

if using_tif:
    ## how to identify the movies: the first many characters of the filenames (important if using_tif)
    fueled_movie_start = '19.3 mg per mL PEG 20000, 6% H2O2, WPF'
    control_movie_start = '19.3 mg per mL PEG 20000, WPC'

    #fueled_file = os.path.join(movie_data_folder, movie_name + ' FUELED.tif')
    #control_file = os.path.join(movie_data_folder, movie_name + ' CONTROL.tif')
else:
    ### Here is the most general usage option for people who know what movie they want to edit.
    ### This doesn't work as well with TIF stacks.
    control_movie_filename = "EBL 23-04-26, 17.3fps, 40x, 1x, 200 & 20000 @ 25 mgmL W O ADDED WATER #1.avi"
    fueled_movie_filename = ".avi"
    
    if doing_fueled:
        fueled_file = os.path.join(movie_data_folder, fueled_movie_filename)
    if doing_control:
        control_file = os.path.join(movie_data_folder, control_movie_filename)

print('Movie data folder: ' + movie_data_folder)

analysis_folder = os.path.join(home, 'Analysis')
movie_analysis_folder = os.path.join(analysis_folder, today, 'Data Taken ' + data_date + ', ' + movie_name)

if (not os.path.exists(movie_analysis_folder)):
    os.makedirs(movie_analysis_folder)
    print("Created movie analysis folder" + movie_analysis_folder)
else:
    print('Using movie analysis folder: ' + movie_analysis_folder)

if doing_control:
    assert os.path.isfile(control_file), f"File does not exist: {control_file}"
if doing_fueled:
    assert os.path.isfile(fueled_file), f"File does not exist: {control_file}"

if 'scaling' not in dir():
    raise NameError('Please set the variable "scaling" in microns per pixel. Use a calibration slide.')

# Open movie files

In [ ]:
# Attempts to open video file and extract fps info

def get_fps_from_avi(avi_file): # avi_file is a file path
    try:
        video = cv2.VideoCapture(avi_file)
        fps = video.get(cv2.CAP_PROP_FPS)
        video.release()
        if fps == 0:
            raise Exception("zero fps")
    except:
        import av
        container = av.open(avi_file)
        fps = container.streams.video[0].average_rate
        container.close()
    return fps

if doing_fueled:
    fps = get_fps_from_avi(fueled_file)
else:
    fps = get_fps_from_avi(control_file)
frametime = 1000/fps    # milliseconds

In [ ]:
fps

In [ ]:
# Declaring functions for channel separation.
# Movies are assumed to have green and red particles.

@pims.pipeline
def red_separation(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    blue = frame[:,:,2]
    return 1 * red + 0 * green + 0 * blue
    
@pims.pipeline
def green_separation(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    blue = frame[:,:,2]
    return 0 * red + 1 * green + 0 * blue

@pims.pipeline
def blue_separation(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    blue = frame[:,:,2]
    return 0 * red + 0 * green + 1 * blue

# bluegreen is good for blue particles
@pims.pipeline
def bluegreen(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    blue = frame[:,:,2]
    return 0 * red + (1/2.0) * green + (1/2.0) * blue

@pims.pipeline
def separation(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    for row in range(len(green)):
        for col in range(len(green[0])):
            if (green[row, col] < red[row, col]):
                green[row, col] = 0
            else:
                red[row, col] = 0
    return green, red

@pims.pipeline
def grayscale(frame):
    red = frame[:,:,0]
    green = frame[:,:,1]
    blue = frame[:,:,2]
    return (1/3.0) * red + (1/3.0) * green + (1/3.0) * blue 
    # It is crucial to divide before adding. If the numbers ever exceed 255, they wrap around to zero.

# Creating folders for channel separation.

channel_separated_folder = os.path.join(movie_analysis_folder, 'Channel Separated Videos')

if (not os.path.exists(data_folder)):
    os.makedirs(data_folder)
    print("Created " + data_folder)
else:
    print('Already exists:', data_folder)
if (not os.path.exists(movie_data_folder)):
    os.makedirs(movie_data_folder)
    print("Created " + movie_data_folder)
else:
    print('Already exists:', movie_data_folder)

if (not os.path.exists(channel_separated_folder)):
    os.makedirs(channel_separated_folder)
    print("Created " + channel_separated_folder)
else:
    print('Already exists:', channel_separated_folder)

In [ ]:
## Thorcam tends to create a series of tiff stacks to create one movie. This concatenates them together.
def open_and_concatenate_tiff_stack(movie_data_folder, filestart='control'):
    print('Opening ' + filestart + ' files from ' + movie_data_folder)
    data = np.concatenate([pims.TiffStack(os.path.join(movie_data_folder, item))
                           for item in np.sort(os.listdir(movie_data_folder)) if item.startswith(filestart)])
    return data

## Open each movie and load into memory    
if using_tif:
    if doing_fueled:
        fueled_data = open_and_concatenate_tiff_stack(movie_data_folder, fueled_movie_start)
    if doing_control:
        control_data = open_and_concatenate_tiff_stack(movie_data_folder, control_movie_start)
else: # avi
    if doing_fueled:
        fueled_data = pims.Video(fueled_file)
    if doing_control:
        control_data = pims.Video(control_file)

### How to analyze only a portion of a video:
## control_data = control_data[167:333]     
        
if colorseparation:
    # Performing channel separation.

    fueled_green = []
    fueled_red = []
    control_green = []
    control_red = []

    # Fueled channel separation. separation() does conditional separation of each pixel based on which
    # RGB component is most prevalent.
    if doing_fueled:
        for frame in fueled_data:
            #green, red = separation(frame)
            #fueled_green.append(green)
            #fueled_red.append(red)
            fueled_green.append(green_separation(frame))
            fueled_red.append(red_separation(frame))
        with tifffile.TiffWriter(os.path.join(channel_separated_folder, 'Fueled Green.tif')) as tif:
            for frame in fueled_green:
                tif.write(frame, contiguous=True)
        with tifffile.TiffWriter(os.path.join(channel_separated_folder, 'Fueled Red.tif')) as tif:
            for frame in fueled_red:
                tif.write(frame, contiguous=True)

    # Control channel separation, save series of channel-separated images.
    if doing_control:
        for frame in control_data:
            #green, red = separation(frame)
            #control_green.append(green)
            #control_red.append(red)
            control_green.append(green_separation(frame))
            control_red.append(red_separation(frame))
        with tifffile.TiffWriter(os.path.join(channel_separated_folder, 'Control Green.tif')) as tif:
            for frame in control_green:
                tif.write(frame, contiguous=True)
        with tifffile.TiffWriter(os.path.join(channel_separated_folder, 'Control Red.tif')) as tif:
            for frame in control_red:
                tif.write(frame, contiguous=True)

    """
    ## Clear memory of the rgb movies. 
    ## The movies will need to be read again for making the trajectory movie. 
    ## Comment this out if you would rather use memory than spend the time.
    if doing_fueled:
        del fueled_data
    if doing_control:
        del control_data
    """

    gc.collect()


    if doing_fueled:
        fueled_green = list(pims.TiffStack(os.path.join(channel_separated_folder, 'Fueled Green.tif')))
        fueled_red = list(pims.TiffStack(os.path.join(channel_separated_folder, 'Fueled Red.tif')))
    if doing_control:
        control_green = list(pims.TiffStack(os.path.join(channel_separated_folder, 'Control Green.tif')))
        control_red = list(pims.TiffStack(os.path.join(channel_separated_folder, 'Control Red.tif')))
else: # not separating green and red? 
    if doing_control:
        control_green = grayscale(control_data) # or please use bluegreen if you are using blue particles ******
        control_red = control_green
    if doing_fueled:
        fueled_green = grayscale(fueled_data)
        fueled_red = fueled_green

In [ ]:
## Show a frame from each movie
if doing_fueled and colorseparation:
    plt.figure()
    plt.imshow(fueled_green[50], cmap='gray')
    plt.title('A frame from Fueled Green')
    
if doing_fueled:
    plt.figure()
    plt.imshow(fueled_red[50], cmap='gray')
    plt.title('A frame from Fueled Red')
    
if doing_control and colorseparation:
    plt.figure()
    plt.imshow(control_green[50], cmap='gray')
    plt.title('A frame from Control Green')
    
if doing_control:
    plt.figure()
    plt.imshow(control_red[50], cmap='gray')
    plt.title('A frame from Control Red')

# Test finding bright spots in ONE frame
You will probably need to adjust settings to use this properly.

In [ ]:
# Find bright spots in a frame of Fueled Green.
# featuresize must be odd.
# Read up on this in the trackpy literature.
# You need to update this if the particle, objective, or camera changes!!

i = 50 # Frame to search in
bright_background = False # Only true if looking for dark spots on a bright background
feature_size = 15 # Change this to find different size particles, can use ImageJ to get approximate feature size?
feature_mass_green = 375
feature_mass_red = 120

if doing_control:
    f1 = tp.locate(control_red[i], diameter=feature_size, invert=bright_background, minmass=feature_mass_red)
else:
    f1 = tp.locate(fueled_green[i], diameter=feature_size, invert=bright_background, minmass=feature_mass_green)

plt.figure()  # make a new figure
if doing_control:
    tp.annotate(f1, control_red[i])
else:
    tp.annotate(f1, fueled_green[i])

tp.subpx_bias(f1)  # for determining if particles bias a pixel (you can check before doing the whole batch)

#del f1

In [ ]:
# Image cropping! We need to see if tracking is working, so we need to zoom in on a few particles
view_number = 10 # number of particles to view here

import random
# pick a few random particles. You can re-run this cell to select more.
rows = random.sample(range(0, len(f1)), view_number)

# The image itself is 1024 rows x 1280 columns # set width of image to display
# width1 = 600 # good for context # commented out.
width2 = 180 # width of image to display. 
             
# edges() is a function for how to calculate the edges of a cropped image so we can zoom in.                     
def edges(x,y,width):
    xlow = int(max(0, x-width/2)) # edges for cropping the image. Don't go below 0!
    xhigh = int(x+width/2) # the upper edge
    ylow = int(max(0,y-width/2))
    yhigh = int(y+width/2)

    # subtract low edges to re-center coordinates
    coords = pd.DataFrame({'x': [x-xlow], 'y': [y-ylow]})  # x: horizontal, y: vertical
    return xlow,xhigh,ylow,yhigh, coords
                     
for row in rows:
    x = f1.iloc[row].x
    y = f1.iloc[row].y # pick out 1 particle x and y coordinates.
    #display(f1.iloc[row])

    #xlow1,xhigh1,ylow1,yhigh1, coords1 = edges(x,y,width1)
    xlow2,xhigh2,ylow2,yhigh2, coords2 = edges(x,y,width2)


    #plt.figure()
    #plt.title('frame ' + str(int(f1.iloc[row].frame)) + ', particle ' + str(f1.index[row]))
    ## slicing: vertical, horizontal.
    #tp.annotate(centroids = coords1, image = control_red[i][ylow1:yhigh1, xlow1:xhigh1]); 


    plt.figure()
    plt.title('frame ' + str(int(f1.iloc[row].frame)) + ', particle ' + str(f1.index[row]))
    tp.annotate(centroids = coords2, 
                image = control_red[i][ylow2:yhigh2, xlow2:xhigh2], 
                plot_style={'markersize': 50} ); # size 2 or 50 is good

# Find bright spots in ALL frames (slow)

In [ ]:
# Now that we have picked out an appropriate featuresize and settings, it's time to go through ALL the frames,
# finding the coordinates of the bright spots in each frame.
# batch already uses a loop to go through frames

time_sec_1 = time.time()
# Elapsed time 1.4 min

In [ ]:
if doing_fueled:
    fueled_tracer_coords = tp.batch(fueled_red, diameter=feature_size, invert=bright_background, minmass=feature_mass_red)
    tp.subpx_bias(fueled_tracer_coords)

In [ ]:
if doing_fueled and  colorseparation:
    fueled_janus_coords = tp.batch(fueled_green, diameter=feature_size, invert=bright_background, minmass=feature_mass_green)
    tp.subpx_bias(fueled_janus_coords)
elif doing_fueled:
    fueled_janus_coords = fueled_tracer_coords

## Check tracking (especially featuresize) using subpixel biax

In [ ]:
if doing_control:
    try:
        control_tracer_coords = tp.batch(control_red, diameter=feature_size, invert=bright_background, minmass=feature_mass_red)
    except ValueError as ve:
        print('Might have had issues with multiple processors. Repeating on just one processor.')
        control_tracer_coords = tp.batch(control_red, diameter=feature_size, invert=bright_background, 
                                         minmass=feature_mass_red, processes=1)
    tp.subpx_bias(control_tracer_coords)

In [ ]:
if doing_control and  colorseparation:
    try:
        control_janus_coords = tp.batch(control_green, diameter=feature_size, invert=bright_background, minmass=feature_mass_green)
    except ValueError as ve:
        print('Might have had issues with multiple processors. Repeating on just one processor.')
        print(ve)
        control_janus_coords = tp.batch(control_green, diameter=feature_size, invert=bright_background, 
                                        minmass=feature_mass_green, processes=1)
    tp.subpx_bias(control_janus_coords)
elif doing_control:
    control_janus_coords = control_tracer_coords

In [ ]:
 
time_sec_2 = time.time()
elapsed_min = (time_sec_2-time_sec_1)/60.0
print("Elapsed time " + str(elapsed_min) + " min" )

# Documentation: http://soft-matter.github.io/trackpy/generated/trackpy.batch.html

# This is an implementation of the Crocker-Grier centroid-finding algorithm.
#  Crocker, J.C., Grier, D.G. http://dx.doi.org/10.1006/jcis.1996.0217

In [ ]:
# Note: These particle numbers count each frame individually, not the trajectories.
if doing_fueled:
    if colorseparation:
        print('Fueled Janus')
        print('Number of Frames: ' + str(len(fueled_green)))
        print('Number of Particles * Frames: ' + str(len(fueled_janus_coords)))
        print()
    print('Fueled Tracer')
    print('Number of Frames: ' + str(len(fueled_red)))
    print('Number of Particles * Frames: ' + str(len(fueled_tracer_coords)))
    print()
if doing_control:
    if colorseparation:
        print('Control Janus')
        print('Number of Frames: ' + str(len(control_green)))
        print('Number of Particles * Frames: ' + str(len(control_janus_coords)))
        print()
    print('Control Tracer')
    print('Number of Frames: ' + str(len(control_red)))
    print('Number of Particles * Frames: ' + str(len(control_tracer_coords)))

# Link Trajectories

In [ ]:
# We have just built a list of coordinates called f_coords where we have seen particles. '
# Now we want to link these together from one frame to the next 
# so we can identify the trajectory for each particle.

# Documentation: http://soft-matter.github.io/trackpy/v0.3.0/generated/trackpy.link.html

# Fastest particle velocity observed in Viva's EPJST paper was about 15 microns/sec. Search
# range should be scalable based on that upper bound.
fastest = 15 # microns / second
memory = 5
search_range = int(round(fastest/(fps * scaling)))


print('Linking trajectories from one frame to the next with a memory of', memory, 'frames.')
if doing_fueled:
    if colorseparation:
        fueled_janus_unfiltered = tp.link_df(f=fueled_janus_coords, search_range=search_range, memory=memory)
    fueled_tracer_unfiltered = tp.link_df(f=fueled_tracer_coords, search_range=search_range, memory=memory)
if doing_control:
    if colorseparation:
        control_janus_unfiltered = tp.link_df(f=control_janus_coords, search_range=search_range, memory=memory)
    control_tracer_unfiltered = tp.link_df(f=control_tracer_coords, search_range=search_range, memory=memory)
"""
if doing_fueled:
    del fueled_janus_coords, fueled_tracer_coords
if doing_control:
    del control_janus_coords, control_tracer_coords
"""
if doing_fueled:
    if colorseparation:
        fueled_janus_unfiltered.to_pickle(movie_analysis_folder + 'fueled_janus_unfiltered.pkl')
    fueled_tracer_unfiltered.to_pickle(movie_analysis_folder + 'fueled_tracer_unfiltered.pkl')
if doing_control:
    if colorseparation:
        control_janus_unfiltered.to_pickle(movie_analysis_folder + 'control_janus_unfiltered.pkl')
    control_tracer_unfiltered.to_pickle(movie_analysis_folder + 'control_tracer_unfiltered.pkl')

print("The search range was {0} pixels.".format(search_range))

In [ ]:
## Plot unfiltered trajectories

if do_slow_stuff:
    print('Trajectories before filtering')
    if doing_fueled:
        if colorseparation:
            print('Fueled Janus')
            fueled_janus_trajectory_plot = tp.plot_traj(fueled_janus_unfiltered,
                                           superimpose = fueled_green[int(len(fueled_green)/2)], label=False)

        print('Fueled Tracer')
        fueled_tracer_trajectory_plot = tp.plot_traj(fueled_tracer_unfiltered,
                                       superimpose = fueled_red[int(len(fueled_red)/2)], label=False)

    if doing_control:
        if colorseparation:
            print('Control Janus')
            control_janus_trajectory_plot = tp.plot_traj(control_janus_unfiltered,
                                           superimpose = control_green[int(len(control_green)/2)], label=False)

        print('Control Tracer')
        control_tracer_trajectory_plot = tp.plot_traj(control_tracer_unfiltered,
                                       superimpose = control_red[int(len(control_red)/2)], label=False)

# Look at tracked particles

In [ ]:
# Image cropping! We need to see if tracking is working, so we need to zoom in on a few particles
view_number = 10 # number of particles to view here

import random
# pick a few random particles. You can re-run this cell to select more.
rows = random.sample(range(0, len(control_tracer_coords)), view_number)

# The image itself is 1024 rows x 1280 columns # set width of image to display
# width1 = 600 # good for context # commented out.
width2 = 180 # width of image to display. 
             
# edges() is a function for how to calculate the edges of a cropped image so we can zoom in.                     
def edges(x,y,width):
    xlow = int(max(0, x-width/2)) # edges for cropping the image. Don't go below 0!
    xhigh = int(x+width/2) # the upper edge
    ylow = int(max(0,y-width/2))
    yhigh = int(y+width/2)

    # subtract low edges to re-center coordinates
    coords = pd.DataFrame({'x': [x-xlow], 'y': [y-ylow]})  # x: horizontal, y: vertical
    return xlow,xhigh,ylow,yhigh, coords
                     
for row in rows:
    x = control_tracer_unfiltered.iloc[row].x
    y = control_tracer_unfiltered.iloc[row].y # pick out one particle x and y coordinates.

    #xlow1,xhigh1,ylow1,yhigh1, coords1 = edges(x,y,width1)
    xlow2,xhigh2,ylow2,yhigh2, coords2 = edges(x,y,width2)
    
    i = control_tracer_unfiltered.iloc[row].frame

    plt.figure()
    plt.title('frame ' + str(int(i)) + ', particle ' + str(control_tracer_unfiltered.iloc[row].particle))
    tp.annotate(centroids = coords2, 
                image = control_red[i][ylow2:yhigh2, xlow2:xhigh2], 
                plot_style={'markersize': 50} ); # size 2 or 50 is good

# Filter trajectories by length

In [ ]:
# Only keep trajectories that last at least this many frames
filter_length = 10
print('Filtering out trajectories that disappear before', filter_length, 'frames.')

if doing_fueled:
    fueled_tracer = tp.filter_stubs(fueled_tracer_unfiltered, filter_length)
    if colorseparation:
        fueled_janus = tp.filter_stubs(fueled_janus_unfiltered, filter_length)
    else:
        fueled_janus = fueled_tracer
if doing_control:
    control_tracer = tp.filter_stubs(control_tracer_unfiltered, filter_length)
    if colorseparation:
        control_janus = tp.filter_stubs(control_janus_unfiltered, filter_length)
    else:
        control_janus = control_tracer

## Save the trajectories
if doing_fueled:
    if colorseparation:
        fueled_janus.to_pickle(movie_analysis_folder + 'fueled_janus.pkl')
    fueled_tracer.to_pickle(movie_analysis_folder + 'fueled_tracer.pkl')
if doing_control:
    if colorseparation:
        control_janus.to_pickle(movie_analysis_folder + 'control_janus.pkl')
    control_tracer.to_pickle(movie_analysis_folder + 'control_tracer.pkl')

# Compare the number of particles in the unfiltered and filtered data.
if doing_fueled:
    if colorseparation:
        print('Fueled Janus')
        print('Before:', fueled_janus_unfiltered['particle'].nunique())
        print('After:', fueled_janus['particle'].nunique())
        metric4_fueled_janus = fueled_janus['particle'].nunique()
        print()
    print('Fueled Tracer')
    print('Before:', fueled_tracer_unfiltered['particle'].nunique())
    print('After:', fueled_tracer['particle'].nunique())
    metric4_fueled_tracer = fueled_tracer['particle'].nunique()
    print()
if doing_control:
    if colorseparation:
        print('Control Janus')
        print('Before:', control_janus_unfiltered['particle'].nunique())
        print('After:', control_janus['particle'].nunique())
        metric4_control_janus = control_janus['particle'].nunique()
        print()
    print('Control Tracer')
    print('Before:', control_tracer_unfiltered['particle'].nunique())
    print('After:', control_tracer['particle'].nunique())
    metric4_control_tracer = control_tracer['particle'].nunique()

# Visualize trajectories

In [ ]:
if do_slow_stuff:
    if doing_fueled:
        if colorseparation:
            print('Fueled Janus')
            fueled_janus_trajectory_plot = tp.plot_traj(fueled_janus,
                                           superimpose = fueled_green[int(len(fueled_green)/2)], label=False)

        print('Fueled Tracer')
        fueled_tracer_trajectory_plot = tp.plot_traj(fueled_tracer,
                                       superimpose = fueled_red[int(len(fueled_red)/2)], label=False)

    if doing_control:
        if colorseparation:
            print('Control Janus')
            control_janus_trajectory_plot = tp.plot_traj(control_janus,
                                           superimpose = control_green[int(len(control_green)/2)], label=False)

        print('Control Tracer')
        control_tracer_trajectory_plot = tp.plot_traj(control_tracer,
                                       superimpose = control_red[int(len(control_red)/2)], label=False)

In [ ]:
try:
    axes().set_aspect('equal', 'datalim') 
except:
    pass

if do_slow_stuff:
    if doing_fueled:
        if colorseparation:
            print('Fueled Janus')
            plt.axis('equal')
            fueled_janus_trajectory_plot = tp.plot_traj(fueled_janus, mpp=scaling)

        print('Fueled Tracer')
        plt.axis('equal')
        fueled_tracer_trajectory_plot = tp.plot_traj(fueled_tracer, mpp=scaling)

    if doing_control:
        if colorseparation:
            print('Control Janus')
            plt.axis('equal')
            control_janus_trajectory_plot = tp.plot_traj(control_janus, mpp=scaling)

        print('Control Tracer')
        plt.axis('equal')
        control_tracer_trajectory_plot = tp.plot_traj(control_tracer, mpp=scaling)

# Drift Subtraction

In [ ]:
time_sec_3 = time.time()
# Elapsed time 0.3 min

# Conduct rotational drift subtraction.

print('Conducting drift subtraction.')
if doing_fueled:
    fueled_janus_nodrift, fueled_tracer_nodrift = drift_subtract(fueled_janus, fueled_tracer)
    if not colorseparation:
        fueled_janus_nodrift = fueled_tracer_nodrift
if doing_control:
    control_janus_nodrift, control_tracer_nodrift = drift_subtract(control_janus, control_tracer)
    if not colorseparation:
        control_janus_nodrift = control_tracer_nodrift


time_sec_4 = time.time()
elapsed_min = (time_sec_4-time_sec_3)/60.0
print("Elapsed time " + str(elapsed_min) + " min" )
    
if doing_fueled:
    if colorseparation:
        fueled_janus_nodrift.to_pickle(movie_analysis_folder + 'fueled_janus_nodrift.pkl')
    fueled_tracer_nodrift.to_pickle(movie_analysis_folder + 'fueled_tracer_nodrift.pkl')
if doing_control:
    if colorseparation:
        control_janus_nodrift.to_pickle(movie_analysis_folder + 'control_janus_nodrift.pkl')
    control_tracer_nodrift.to_pickle(movie_analysis_folder + 'control_tracer_nodrift.pkl')
    
plt.tight_layout()

In [ ]:
## View translational drift (though we use rotational drift for drift
## subtraction)
smoothing = 15 # frames

if doing_control:
    drift_control_tracer = tp.compute_drift(control_tracer, smoothing)*scaling
    if colorseparation:
        drift_control_janus = tp.compute_drift(control_janus, smoothing)*scaling
    else:
        drift_control_janus = drift_control_tracer

    # Convert from frames to seconds
    drift_control_janus.index = drift_control_janus.index /fps
    drift_control_tracer.index = drift_control_tracer.index /fps
    drift_control_janus.index.name = 'time (s)'
    drift_control_tracer.index.name = 'time (s)'

    if colorseparation:
        plt.figure()
        drift_control_janus.plot(grid=False)
        plt.title('Cumulative translational drift of control Janus\n' + movie_name + '\n')
        plt.ylabel('Ensemble translational drift ($\mathrm{\mu}$m)')
        #plt.xlabel('seconds')

    plt.figure()
    drift_control_tracer.plot(grid=False)
    plt.title('Cumulative translational drift of control tracers\n' + movie_name + '\n')
    plt.ylabel('Ensemble translational drift ($\mathrm{\mu}$m)')

    if colorseparation:
        motion_control_janus = drift_control_janus.diff()
        velocity_control_janus_df = (drift_control_janus.diff()/ (1/fps))
        velocity_control_janus_df.plot()
        plt.ylabel("Control Janus ensemble mean velocity ($\mathrm{\mu}$m/s)")
        plt.title('Smoothing: ' + str(smoothing) + " frames")


    motion_control_tracer = drift_control_tracer.diff()
    velocity_control_tracer_df = (drift_control_tracer.diff()/ (1/fps))
    if colorseparation:
        velocity_control_tracer_df.plot()
        plt.ylabel("Control tracer ensemble mean velocity ($\mathrm{\mu}$m/s)")
        plt.title('Smoothing: ' + str(smoothing) + " frames")

    ## Add speed to dataframes
    if colorseparation:
        velocity_control_janus_df['speed'] = sqrt(velocity_control_janus_df.x**2 + velocity_control_janus_df.y**2)
    velocity_control_tracer_df['speed'] = sqrt(velocity_control_tracer_df.x**2 + velocity_control_tracer_df.y**2)

    if colorseparation:
        plt.figure()
        velocity_control_janus_df.speed.hist(bins=50, label="control janus")
        plt.xlabel("Lateral translational drift speed ($\mathrm{\mu}$m/s)")
        plt.ylabel("Occurrences")
        plt.title("control janus")

    plt.figure()
    velocity_control_tracer_df.speed.hist(bins=50, label="control tracer")
    plt.xlabel("Lateral translational drift speed ($\mathrm{\mu}$m/s)")
    plt.ylabel("Occurrences")
    plt.title( "control tracer, $\Delta t =$" + str(frametime) + ' ms' )
    
    max_cartesian_drift_control_tracer_um = max(abs(drift_control_tracer).max())
    print('The maximum drift (in um) in x and y was:')
    display(abs(drift_control_tracer).max() );

if doing_fueled:
    drift_fueled_janus = tp.compute_drift(fueled_janus, smoothing)*scaling
    drift_fueled_tracer = tp.compute_drift(fueled_tracer, smoothing)*scaling

    # Convert from frames to seconds
    drift_fueled_janus.index = drift_fueled_janus.index /fps
    drift_fueled_tracer.index = drift_fueled_tracer.index /fps
    drift_fueled_janus.index.name = 'time (s)'
    drift_fueled_tracer.index.name = 'time (s)'

    plt.figure()
    drift_fueled_janus.plot(grid=False)
    plt.title('Translational drift of fueled Janus\n' + movie_name + '\n')
    plt.ylabel('Ensemble translational drift ($\mathrm{\mu}$m)')

    plt.figure()
    drift_fueled_tracer.plot(grid=False)
    plt.title('Translational drift of fueled tracers\n' + movie_name + '\n')
    plt.ylabel('Ensemble translational drift ($\mathrm{\mu}$m)')

    motion_fueled_janus = drift_fueled_janus.diff()
    velocity_fueled_janus_df = (drift_fueled_janus.diff()/ (1/fps))
    velocity_fueled_janus_df.plot()
    plt.ylabel("fueled Janus ensemble mean velocity ($\mathrm{\mu}$m/s)")
    plt.title('Smoothing: ' + str(smoothing) + " frames")

    motion_fueled_tracer = drift_fueled_tracer.diff()
    velocity_fueled_tracer_df = (drift_fueled_tracer.diff()/ (1/fps))
    velocity_fueled_tracer_df.plot()
    plt.ylabel("fueled tracer ensemble mean velocity ($\mathrm{\mu}$m/s)")
    plt.title('Smoothing: ' + str(smoothing) + " frames")

    ## Add speed to dataframes
    velocity_fueled_janus_df['speed'] = sqrt(velocity_fueled_janus_df.x**2 + velocity_fueled_janus_df.y**2)
    velocity_fueled_tracer_df['speed'] = sqrt(velocity_fueled_tracer_df.x**2 + velocity_fueled_tracer_df.y**2)

    plt.figure()
    velocity_fueled_janus_df.speed.hist(bins=50, label="fueled janus")
    plt.xlabel("Lateral translational drift speed ($\mathrm{\mu}$m/s)")
    plt.ylabel("Occurrences")
    plt.title("fueled janus")

    plt.figure()
    velocity_fueled_tracer_df.speed.hist(bins=50, label="fueled tracer")
    plt.xlabel("Lateral translational drift speed ($\mathrm{\mu}$m/s)")
    plt.ylabel("Occurrences")
    plt.title("fueled tracer")

In [ ]:
if do_slow_stuff:
    if doing_fueled:
        plt.axis('equal')
        plt.title(movie_name + '\n Fueled Janus trajectories with drift subtraction\n')
        ax = tp.plot_traj(fueled_janus_nodrift, legend=False)

    if doing_fueled:
        plt.axis('equal')
        plt.title(movie_name + '\n Fueled Tracer trajectories with drift subtraction\n')
        ax = tp.plot_traj(fueled_tracer_nodrift, legend=False)

    if doing_control and colorseparation:
        plt.axis('equal')
        plt.title(movie_name + '\n Control Janus trajectories with drift subtraction\n')
        ax = tp.plot_traj(control_janus_nodrift, legend=False)

    if doing_control:
        plt.axis('equal')
        plt.title(movie_name + '\n Control Tracer trajectories with drift subtraction\n')
        ax = tp.plot_traj(control_tracer_nodrift, legend=False)

# Calculate MSDs

In [ ]:
start = time.time()

if doing_fueled:
    fueled_janus_imsd = tp.imsd(fueled_janus_nodrift, mpp=scaling, fps=fps, max_lagtime=1000)
    fueled_tracer_imsd = tp.imsd(fueled_tracer_nodrift, mpp=scaling, fps=fps, max_lagtime=1000)
if doing_control:
    control_janus_imsd = tp.imsd(control_janus_nodrift, mpp=scaling, fps=fps, max_lagtime=1000)
    control_tracer_imsd = tp.imsd(control_tracer_nodrift, mpp=scaling, fps=fps, max_lagtime=1000)
    
end = time.time()
    
print('Calculating iMSD took', end-start, 'sec.')

In [ ]:
plt.plot(control_tracer_imsd);
plt.ylabel("Mean square displacement [um^2]");
plt.xlabel("Lag time [s]");
plt.title('iMSD on linear axes')

print("It's normal for this to look messy for random motion.")

In [ ]:
if colorseparation:
    fig, ax = plt.subplots()
    if doing_fueled:
        plt.plot(fueled_janus_imsd, 'r-', alpha=0.15)
    if doing_control:
        plt.plot(control_janus_imsd, '.', alpha=0.05)

    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
           xlabel='lag time $\Delta{}t$ [s]')
    ax.set_xscale('log')
    ax.set_yscale('log')
    fig.set_size_inches(3,3)
    plt.title(movie_name + '\nJanus iMSD, drift-subtracted');

In [ ]:
fig, ax = plt.subplots()
if doing_fueled:
    ax.plot(fueled_tracer_imsd, 'r-', alpha=0.15)
if doing_control:
    ax.plot(control_tracer_imsd, '.', alpha=0.05)

ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $\Delta{}t$ [s]')
ax.set_xscale('log')
ax.set_yscale('log')
fig.set_size_inches(3,3)
plt.title(movie_name + '\nTracer iMSD, drift-subtracted');

In [ ]:
if doing_fueled:
    fueled_janus_imsd_drifty = tp.imsd(fueled_janus, mpp=scaling, fps=fps, max_lagtime=1000)
    fueled_tracer_imsd_drifty = tp.imsd(fueled_tracer, mpp=scaling, fps=fps, max_lagtime=1000)
if doing_control:
    control_janus_imsd_drifty = tp.imsd(control_janus, mpp=scaling, fps=fps, max_lagtime=1000)
    control_tracer_imsd_drifty = tp.imsd(control_tracer, mpp=scaling, fps=fps, max_lagtime=1000)

In [ ]:
if colorseparation:
    fig, ax = plt.subplots()
    if doing_fueled:
        ax.plot(fueled_janus_imsd_drifty, 'r-', alpha=0.15)
    if doing_control:
        ax.plot(control_janus_imsd_drifty, '.', alpha=0.05)

    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
           xlabel='lag time $\Delta{}t$ [s]')
    ax.set_xscale('log')
    ax.set_yscale('log')
    fig.set_size_inches(3,3)
    plt.title(movie_name + '\nJanus iMSD, not drift-subtracted');

In [ ]:
fig, ax = plt.subplots()
if doing_fueled:
    ax.plot(fueled_tracer_imsd_drifty, 'r-', alpha=0.15)
if doing_control:
    ax.plot(control_tracer_imsd_drifty, '.', alpha=0.05)

ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $\Delta{}t$ [s]')
ax.set_xscale('log')
ax.set_yscale('log')
fig.set_size_inches(3,3)
plt.title(movie_name + '\nTracer iMSD, not drift-subtracted');


print("Tell Viva if there are two different populations.")

## Calculate eMSDs

In [ ]:
emsd_max_lagtime = 100
if doing_fueled:
    fueled_janus_emsd = tp.emsd(fueled_janus_nodrift, mpp=scaling, fps=fps, max_lagtime = emsd_max_lagtime).replace(0, np.nan).dropna()
    fueled_tracer_emsd = tp.emsd(fueled_tracer_nodrift, mpp=scaling, fps=fps, max_lagtime = emsd_max_lagtime).replace(0, np.nan).dropna()
if doing_control:
    control_janus_emsd = tp.emsd(control_janus_nodrift, mpp=scaling, fps=fps, max_lagtime = emsd_max_lagtime).replace(0, np.nan).dropna()
    control_tracer_emsd = tp.emsd(control_tracer_nodrift, mpp=scaling, fps=fps, max_lagtime = emsd_max_lagtime).replace(0, np.nan).dropna()

## Fit powerlaw to eMSD
### Filtering out static localization error at small displacements.

In [ ]:
### Filter out small displacements from the curvefit.
threshold = 0.1 ## Might need to adjust this!

In [ ]:
fig, ax1 = plt.subplots()

if doing_fueled:
    fueled_tracer_emsd_filtered = fueled_tracer_emsd[fueled_tracer_emsd >= threshold]
    fueled_tracer_fit = tp.utils.fit_powerlaw(fueled_tracer_emsd_filtered, plot = False)
    
    # Displaying 2 points of the curvefit is sufficient because it appears as a straight line
    fueled_tracer_fit_x = [1/fps, (1/fps) * len(fueled_tracer_emsd)]
    fueled_tracer_fit_y = [fueled_tracer_fit.A[0] * np.power(fueled_tracer_fit_x[0], fueled_tracer_fit.n[0]),
                         fueled_tracer_fit.A[0] * np.power(fueled_tracer_fit_x[1], fueled_tracer_fit.n[0])]

if doing_control:
    control_tracer_emsd_filtered = control_tracer_emsd[control_tracer_emsd >= threshold]
    control_tracer_fit = tp.utils.fit_powerlaw(control_tracer_emsd_filtered, plot = False)
    
    # Displaying 2 points of the curvefit is sufficient because it appears as a straight line
    control_tracer_fit_x = [1/fps, (1/fps) * len(control_tracer_emsd)]
    control_tracer_fit_y = [control_tracer_fit.A[0] * np.power(control_tracer_fit_x[0], control_tracer_fit.n[0]),
                         control_tracer_fit.A[0] * np.power(control_tracer_fit_x[1], control_tracer_fit.n[0])]

if doing_fueled:
    fueled_tracer_emsd.plot(loglog=True, figsize = [3,3], style = 'r.',  grid=False, ax=ax1)
    plt.plot(fueled_tracer_fit_x, fueled_tracer_fit_y, 'r')

if doing_control:
    control_tracer_emsd.plot(loglog=True, figsize = [3,3], style = 'b.',  grid=False, ax=ax1)
    plt.plot(control_tracer_fit_x, control_tracer_fit_y, 'b')

ax1.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $\Delta{}t$ [s]')
plt.title(movie_name + '\nTracer eMSD, drift-subtracted')

plt.savefig(os.path.join(movie_analysis_folder, 'tracer_eMSDs.png'))

In [ ]:
if colorseparation:

    fig, ax1 = plt.subplots()


    if doing_fueled:
        fueled_janus_emsd_filtered = fueled_janus_emsd[fueled_janus_emsd >= threshold]
        fueled_janus_fit = tp.utils.fit_powerlaw(fueled_janus_emsd_filtered, plot = False)

        # Displaying 2 points of the curvefit is sufficient because it appears as a straight line
        fueled_janus_fit_x = [1/fps, (1/fps) * len(fueled_janus_emsd)]
        fueled_janus_fit_y = [fueled_janus_fit.A[0] * np.power(fueled_janus_fit_x[0], fueled_janus_fit.n[0]),
                             fueled_janus_fit.A[0] * np.power(fueled_janus_fit_x[1], fueled_janus_fit.n[0])]

    if doing_control:
        control_janus_emsd_filtered = control_janus_emsd[control_janus_emsd >= threshold]
        control_janus_fit = tp.utils.fit_powerlaw(control_janus_emsd_filtered, plot = False)

        # Displaying 2 points of the curvefit is sufficient because it appears as a straight line
        control_janus_fit_x = [1/fps, (1/fps) * len(control_janus_emsd)]
        control_janus_fit_y = [control_janus_fit.A[0] * np.power(control_janus_fit_x[0], control_janus_fit.n[0]),
                             control_janus_fit.A[0] * np.power(control_janus_fit_x[1], control_janus_fit.n[0])]

    if doing_fueled:
        fueled_janus_emsd.plot(loglog=True, figsize = [3,3], style = 'r.',  grid=False, ax=ax1)
        plt.plot(fueled_janus_fit_x, fueled_janus_fit_y, 'r')

    if doing_control:
        control_janus_emsd.plot(loglog=True, figsize = [3,3], style = 'b.',  grid=False, ax=ax1)
        plt.plot(control_janus_fit_x, control_janus_fit_y, 'b')

    ax1.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $\Delta{}t$ [s]')
    plt.title(movie_name + '\nJanus eMSD, drift-subtracted')

    plt.savefig(os.path.join(movie_analysis_folder, 'janus_eMSDs.png'))

In [ ]:
control_tracer_emsd.to_csv(os.path.join(movie_analysis_folder, movie_name + '.csv'))

In [ ]:
fig, ax1 = plt.subplots()

if colorseparation:
    if doing_fueled:
        fueled_janus_emsd.plot(loglog=True, figsize = [3,3], style = 'k.',  grid=False, ax=ax1, label="fueled Janus")
        plt.plot(fueled_janus_fit_x, fueled_janus_fit_y, 'k')

    if doing_control:
        control_janus_emsd.plot(loglog=True, figsize = [3,3], style = 'g.',  grid=False, ax=ax1, label="control Janus")
        plt.plot(control_janus_fit_x, control_janus_fit_y, 'g')

if doing_fueled:
    fueled_tracer_emsd.plot(loglog=True, figsize = [3,3], style = 'r.',  grid=False, ax=ax1, label="fueled Tracer")
    plt.plot(fueled_tracer_fit_x, fueled_tracer_fit_y, 'r')

if doing_control:
    control_tracer_emsd.plot(loglog=True, figsize = [3,3], style = 'b.',  grid=False, ax=ax1, label="control Tracer")
    plt.plot(control_tracer_fit_x, control_tracer_fit_y, 'b')

ax1.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $\Delta{}t$ [s]')
plt.title(movie_name + 
          '\nDrift-subtracted eMSD'+
          '\neMSD Threshold = ' + str(threshold) + r' $\mu m^2$')

plt.legend()


if colorseparation:
    savestr = 'janus+tracer'
else:
    savestr = 'particle'
    
savefile = os.path.join(movie_analysis_folder, savestr + '_eMSDs,thresholded_fit.pdf')
print('Saved to:',savefile )

plt.savefig(savefile)


In [ ]:
if doing_fueled:
    if colorseparation:
        print('Fueled Janus: n =', fueled_janus_fit.values[0,0], 'A =' , fueled_janus_fit.values[0,1])
    print('Fueled Tracers: n =', fueled_tracer_fit.values[0,0], 'A =' , fueled_tracer_fit.values[0,1])
if doing_control:
    if colorseparation:
        print('Control Janus: n =', control_janus_fit.values[0,0], 'A =' , control_janus_fit.values[0,1])
    print('Control Tracers: n =', control_tracer_fit.values[0,0], ', A =' , control_tracer_fit.values[0,1])

if doing_fueled:
    if colorseparation:
        fueled_janus_emsd.to_csv(os.path.join(movie_analysis_folder, "fueled_janus_emsd.csv"))
    fueled_tracer_emsd.to_csv(os.path.join(movie_analysis_folder, "fueled_janus_emsd.csv"))
if doing_control:
    if colorseparation:
        control_janus_emsd.to_csv(os.path.join(movie_analysis_folder, "fueled_janus_emsd.csv"))
    control_tracer_emsd.to_csv(os.path.join(movie_analysis_folder, "fueled_janus_emsd.csv"))

# Gamma calculation
## following Diego Krapf

In [ ]:
verbose = False
minimumlength = 300 # set to a number that retains a reasonable number of particle trajectories

# filter stubs to remove short trajectories.
df = tp.filter_stubs(control_tracer_nodrift, threshold=minimumlength)

print(r'Calculating gamma with particle trajectories longer than', minimumlength, 'frames')

freqs_rad_per_sec = np.logspace(-1, np.log10(fps/2), 30) # Use Nyquist frequency fps/2 as maximum

#particle = df.particle.values[0]
# print('Using a minimum trajectory length of', minimumlength, 'frames, we analyze', df.particle.nunique(), 'particle trajectories.')

print('Using a minimum trajectory length of', minimumlength, 'frames, we analyze', df.particle.nunique(), 'particle trajectories.')

#  initialize
Pxxes = np.zeros((len(freqs_rad_per_sec), df.particle.nunique()), dtype = "float64") 
Pyyes = np.zeros((len(freqs_rad_per_sec), df.particle.nunique()), dtype = "float64") 

if verbose:
    print("Initialized array of size:", (df.particle.nunique(), len(freqs_rad_per_sec)))

for (particle,reindex_particle) in zip(df.particle.unique(), range(df.particle.nunique())):
    x = ((df[df.particle == particle]).x)
    y = ((df[df.particle == particle]).y)    
    xminusx0 = x - x.values[0]
    yminusy0 = y - y.values[0]
    frames = x.index.astype(int)
    #fps = combined_df.fps[i]
    times_in_sec = frames/fps

    # Calculate Probability Spectral Density for data with missing times.
    Pxx_den = scipy.signal.lombscargle(times_in_sec, xminusx0, freqs_rad_per_sec)
    Pyy_den = scipy.signal.lombscargle(times_in_sec, yminusy0, freqs_rad_per_sec)

    
    plt.loglog(freqs_rad_per_sec, Pxx_den,alpha = .1 )

    if verbose:
        print('Particle', particle, 'reindexed as', reindex_particle)
        print('length of samples:', len(xminusx0))
        display(x)
        print('Pxx_den:',Pxx_den)
    Pxxes[:, reindex_particle] = Pxx_den
    Pyyes[:, reindex_particle] = Pyy_den

    
plt.loglog(freqs_rad_per_sec, np.mean(Pxxes, axis=1), 'k')
#plt.ylim(-1, 250)
plt.xlabel("Frequency (rad/s)")
plt.ylabel("x PSD ($\mathrm{\mu}$m$^2$ / Hz)")
#plt.title(combined_df['control_tracer.pkl'].values[i]);
#plt.title('suspected subdiffusion')


# Convert 2-sided to 1-sided psd
pxx = Pxxes / 2
pyy = Pyyes / 2

# Calculate gamma
sigmax = np.std(pxx, axis=1) # standard deviation across many particle trajectories
mux = np.mean(pxx, axis=1)
gammax = sigmax / mux  # standard deviation / mean of PSD, as a function of frequency.

sigmay = np.std(pyy, axis=1)
muy = np.mean(pyy, axis=1)
gammay = sigmay / muy 

plt.figure()
plt.loglog(freqs_rad_per_sec, mux, label = r"mean PSD x")
plt.loglog(freqs_rad_per_sec, muy, label = r"mean PSD y")


plt.loglog(freqs_rad_per_sec, sigmax, label = r"$\sigma_x$")
plt.loglog(freqs_rad_per_sec, sigmay, label = r"$\sigma_y$")
plt.legend()
plt.xlabel('Frequency [rad/s]')
plt.ylabel(r'PSD (mean and stdev) [$\mathrm{\mu m^2/Hz}]$');
#plt.title(desc)

plt.figure()

plt.semilogx(freqs_rad_per_sec, gammax, label=r"$\gamma_x$")
plt.semilogx(freqs_rad_per_sec, gammay, label=r"$\gamma_y$")
plt.legend()
plt.xlabel('Frequency (rad/s)')
plt.ylabel('Gamma');
#plt.title(desc)

print('Total video length:', df.frame.max() / fps, 'sec')

del df

# Probability distribution function (called PDF or van Hove)
## and excess kurtosis of van Hove

In [ ]:
## Settings probability distribution function

vanhove_lagtime = 15 # Measured in frames, adjust as desired.
vanhove_max_x = 5 # Adjust graph width as desired
binwidth = scaling # use the pixel size for the binwidth


# Define functions
"""
manualnorm()

Normalizes histogram data so that the sum of probabilities is one.

@param histdata - the starting histogram

@return the normalized histogram of probabilites
"""
def manualnorm(histdata):
     return (1/(histdata.sum()*binwidth))*histdata

    
"""
gaussian()

Outputs f(x) where f is a Gaussian curve.

@param x - the independent variable
@param a - Gaussian amplitude
@param b - Gaussian center
@param c - Gaussian width

@return f(x)
"""
def gaussian(x,a,b,c):
    return a*(np.exp(-((x-b)**2)/(2*(c**2))))

In [ ]:
## reorder tracks by frame number
if doing_fueled:
    tracksbyframe_fueled_tracer = fueled_tracer_nodrift.set_index(['frame','particle'])['x'].unstack()
if doing_control:
    tracksbyframe_control_tracer = control_tracer_nodrift.set_index(['frame','particle'])['x'].unstack()
binsequence=np.arange(-vanhove_max_x,vanhove_max_x,binwidth)
## This makes a histogram of how far the particles traveled
#    You can choose the lagtime
if doing_fueled:
    trackshist_fueled_tracer_lg6 = tp.motion.vanhove(tracksbyframe_fueled_tracer, lagtime = vanhove_lagtime, mpp =
    scaling,bins = binsequence, ensemble = True)
if doing_control:
    trackshist_control_tracer_lg6 = tp.motion.vanhove(tracksbyframe_control_tracer, lagtime = vanhove_lagtime, mpp =
    scaling,bins = binsequence, ensemble = True)

if doing_fueled:
    trackshist_fueled_tracer_lg6_norm = manualnorm(trackshist_fueled_tracer_lg6)
if doing_control:
    trackshist_control_tracer_lg6_norm = manualnorm(trackshist_control_tracer_lg6)
if doing_fueled:
    trackshist_fueled_tracer_lg6_norm.plot(color = 'red', label="Tracers with fuel")
if doing_control:
    trackshist_control_tracer_lg6_norm.plot(color = 'blue', label="Tracers")
plt.title(movie_name + '\nTracer Vanhove with lagtime = ' + str(vanhove_lagtime))
plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
plt.xlabel('$\Delta$ x ($\mu m$)');
plt.legend()

In [ ]:
if colorseparation:
    ## reorder tracks by frame number
    if doing_fueled:
        tracksbyframe_fueled_janus = fueled_janus_nodrift.set_index(['frame','particle'])['x'].unstack()
    if doing_control:
        tracksbyframe_control_janus = control_janus_nodrift.set_index(['frame','particle'])['x'].unstack()
    binsequence=np.arange(-vanhove_max_x,vanhove_max_x,binwidth)
    ## This makes a histogram of how far the particles traveled
    #    You can choose the lagtime
    if doing_fueled:
        trackshist_fueled_janus_lg6 = tp.motion.vanhove(tracksbyframe_fueled_janus, lagtime = vanhove_lagtime, mpp =
        scaling,bins = binsequence, ensemble = True)
    if doing_control:
        trackshist_control_janus_lg6 = tp.motion.vanhove(tracksbyframe_control_janus, lagtime = vanhove_lagtime, mpp =
        scaling,bins = binsequence, ensemble = True)
        
        
    if doing_fueled:
        trackshist_fueled_janus_lg6_norm = manualnorm(trackshist_fueled_janus_lg6)
    if doing_control:
        trackshist_control_janus_lg6_norm = manualnorm(trackshist_control_janus_lg6)
    if doing_fueled:
        trackshist_fueled_janus_lg6_norm.plot(color = 'red')
    if doing_control:
        trackshist_control_janus_lg6_norm.plot(color = 'blue')
    plt.title(movie_name + '\nJanus Vanhove with lagtime = ' + str(vanhove_lagtime)
              + ' frames\nRed = Fueled, Blue = Control')

    plt.legend()
    plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
    plt.xlabel('$\Delta$ x ($\mu$m)');

In [ ]:
# Curve fit a Gaussian to the Vanhoves
if doing_fueled:
    fueled_janus_params, fueled_janus_covm = sco.curve_fit(gaussian, trackshist_fueled_janus_lg6_norm.index,
                                                          trackshist_fueled_janus_lg6_norm.values)
    fueled_janus_yfit = gaussian(trackshist_fueled_janus_lg6_norm.index, fueled_janus_params[0],
                                 fueled_janus_params[1], fueled_janus_params[2])

    trackshist_fueled_janus_lg6_norm.plot(style = 'r.')
    plt.plot(trackshist_fueled_janus_lg6_norm.index, fueled_janus_yfit, color = 'b')

    plt.yscale('log')
    plt.title(movie_name + '\nFueled Janus Vanhove with lagtime = ' + str(vanhove_lagtime) + ' frames,\nwith Gaussian fit')
    plt.legend()
    plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
    plt.xlabel('$\Delta$ x ($\mu m$)')

    # If you want to change the x and y limits:
    # ylim(ymin=0.00003, ymax=10)
    xlim(-vanhove_max_x,vanhove_max_x)

In [ ]:
# Curve fit a Gaussian to the Vanhoves
if doing_fueled:
    fueled_tracer_params, fueled_tracer_covm = sco.curve_fit(gaussian, trackshist_fueled_tracer_lg6_norm.index,
                                                          trackshist_fueled_tracer_lg6_norm.values)
    fueled_tracer_yfit = gaussian(trackshist_fueled_tracer_lg6_norm.index, fueled_tracer_params[0],
                                 fueled_tracer_params[1], fueled_tracer_params[2])

    trackshist_fueled_tracer_lg6_norm.plot(style = 'r.')
    plt.plot(trackshist_fueled_tracer_lg6_norm.index, fueled_tracer_yfit, color = 'b')

    plt.yscale('log')
    plt.title(movie_name + '\nFueled Tracer Vanhove with lagtime = ' + str(vanhove_lagtime) + ' frames,\nwith Gaussian fit')
    plt.legend()
    plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
    plt.xlabel('$\Delta$ x ($\mu m$)')

In [ ]:
# Curve fit a Gaussian to the Vanhoves
if doing_control and colorseparation:
    control_janus_params, control_janus_covm = sco.curve_fit(gaussian, trackshist_control_janus_lg6_norm.index,
                                                                trackshist_control_janus_lg6_norm.values)
    control_janus_yfit = gaussian(trackshist_control_janus_lg6_norm.index, control_janus_params[0],
                                   control_janus_params[1], control_janus_params[2])

    trackshist_control_janus_lg6_norm.plot(style = 'r.', label = 'Janus')
    plt.plot(trackshist_control_janus_lg6_norm.index, control_janus_yfit, color = 'b', label = 'fit' )

    plt.yscale('log')
    plt.title(movie_name + '\nControl Janus Vanhove with lagtime = ' + str(vanhove_lagtime) + ' frames,\nwith Gaussian fit')
    plt.legend()
    plt.ylabel('P($\Delta$ x) (micron)$^{-1}$')
    plt.xlabel('$\Delta$ x (micron)')
    plt.ylim(ymin = 1e-5)

In [ ]:
# Curve fit a Gaussian to the Vanhoves
if doing_control:
    control_tracer_params, control_tracer_covm = sco.curve_fit(gaussian, trackshist_control_tracer_lg6_norm.index,
                                                          trackshist_control_tracer_lg6_norm.values)
    control_tracer_yfit = gaussian(trackshist_control_tracer_lg6_norm.index, control_tracer_params[0],
                                 control_tracer_params[1], control_tracer_params[2])

    trackshist_control_tracer_lg6_norm.plot(style = 'r.', label="Tracers, not fueled")
    plt.plot(trackshist_control_tracer_lg6_norm.index, control_tracer_yfit, color = 'b', label="fit to unfueled tracers")

    plt.yscale('log')
    plt.title(movie_name + '\nControl Tracer Vanhove with lagtime = ' + str(vanhove_lagtime) + ' frames,\nwith Gaussian fit')
    plt.legend()
    plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
    plt.xlabel('$\Delta$ x ($\mu m$)')
    plt.ylim(ymin = 1e-5)

In [ ]:
if colorseparation:
    if doing_fueled:
        trackshist_fueled_janus_lg6_norm.plot(style = 'r.', label = 'Fueled Janus')
        plt.plot(trackshist_fueled_janus_lg6_norm.index, fueled_janus_yfit, color = 'r')
    if doing_control:
        trackshist_control_janus_lg6_norm.plot(style = 'b.', label = 'Control Janus')
        plt.plot(trackshist_control_janus_lg6_norm.index, control_janus_yfit, color = 'b')
if doing_fueled:
    trackshist_fueled_tracer_lg6_norm.plot(style = 'g.', label = 'Fueled Tracer')
    plt.plot(trackshist_fueled_tracer_lg6_norm.index, fueled_tracer_yfit, color = 'g')
if doing_control:
    trackshist_control_tracer_lg6_norm.plot(style = 'y.', label = 'Control Tracer')
    plt.plot(trackshist_control_tracer_lg6_norm.index, control_tracer_yfit, color = 'y')

plt.yscale('log')
plt.title(movie_name + '\nVanhove with lagtime = ' + str(vanhove_lagtime) + ' frames,\nwith Gaussian fit')
plt.legend()
plt.ylabel('P($\Delta$ x) ($\mu m^{-1}$)')
plt.xlabel('$\Delta$ x ($\mu m$)')
plt.ylim(ymin=1e-6)

In [ ]:
print('EXCESS KURTOSIS')
if doing_fueled:
    print('Fueled Janus: ' + str(trackshist_fueled_janus_lg6_norm.kurtosis()))
    metric3_fueled_janus = str(trackshist_fueled_janus_lg6_norm.kurtosis())
    print('Fueled Tracer: ' + str(trackshist_fueled_tracer_lg6_norm.kurtosis()))
    metric3_fueled_tracer = str(trackshist_fueled_tracer_lg6_norm.kurtosis())
if doing_control:
    if colorseparation:
        print('Control Janus: ' + str(trackshist_control_janus_lg6_norm.kurtosis()))
        metric3_control_janus = str(trackshist_control_janus_lg6_norm.kurtosis())
    print('Control Tracer: ' + str(trackshist_control_tracer_lg6_norm.kurtosis()))
    metric3_control_tracer = str(trackshist_control_tracer_lg6_norm.kurtosis())

# Display summary numbers for spreadsheet

In [ ]:
if colorseparation:
    if doing_fueled:
        print("fueled janus n: {0}".format(fueled_janus_fit.values[0,0]))
        print("fueled janus A: {0}".format(fueled_janus_fit.values[0,1]))
        print("fueled janus EXCESS KURTOSIS: {0}".format(str(metric3_fueled_janus)))
        print("fueled janus NUMBER OF PARTICLES AFTER FILTERING: {0}".format(metric4_fueled_janus))
    if doing_control:
        print("control janus n: {0}".format(control_janus_fit.values[0,0]))
        print("control janus A: {0}".format(control_janus_fit.values[0,1]))
        print("control janus EXCESS KURTOSIS: {0}".format(str(metric3_control_janus)))
        print("control janus NUMBER OF PARTICLES AFTER FILTERING: {0}".format(metric4_control_janus))

if doing_fueled:
    print("fueled tracer n: {0}".format(fueled_tracer_fit.values[0,0]))
    print("fueled tracer A: {0}".format(fueled_tracer_fit.values[0,1]))
    print("fueled tracer EXCESS KURTOSIS: {0}".format(str(metric3_fueled_tracer)))
    print("fueled tracer NUMBER OF PARTICLES AFTER FILTERING: {0}".format(metric4_fueled_tracer))
if doing_control:
    print("control tracer n: {0}".format(control_tracer_fit.values[0,0]))
    print("control tracer A: {0}".format(control_tracer_fit.values[0,1]))
    print("control tracer EXCESS KURTOSIS: {0}".format(str(metric3_control_tracer)))
    print("control tracer NUMBER OF PARTICLES AFTER FILTERING: {0}".format(metric4_control_tracer))
    print("control tracer MAXIMUM DRIFT (um):", max_cartesian_drift_control_tracer_um)

# Observe tracked trajectories

In [ ]:
### This section is for making the movie trajectories. 
# These serve two essential purposes:
# (1) Convincing us that we tracked the particles. 
# (2) Convincing other people that we tracked the particles.

# Assumes all videos have the same pixel length and width
if doing_control:
    (y_size,x_size) = control_green[0].shape
else:
    (y_size,x_size) = fueled_green[0].shape

new_movie_fps = 15   # doesn't have to be the same as fps
trajectory_folder = os.path.join(home, 'Trajectory')
movie_trajectory_folder = os.path.join(trajectory_folder, today, movie_name)

if (not os.path.exists(trajectory_folder)):
    os.makedirs(trajectory_folder)
    print("Created " + trajectory_folder)
if (not os.path.exists(os.path.join(trajectory_folder, today))):
    os.makedirs(os.path.join(trajectory_folder, today))
    print("Created " + os.path.join(trajectory_folder, today))
if (not os.path.exists(movie_trajectory_folder)):
    os.makedirs(movie_trajectory_folder)
    print("Created " + movie_trajectory_folder)
    
gc.collect();

In [ ]:
## Set constants

## Aesthetics
scalebar_greyscale = '0.5' # a higher number is more white. Choose something that contrasts with your movie.
scalebar_transparency = 0.75
trajectory_transparency = 0.4  ## 0 is completely transparent and 1 is completely opaque
particle_transparency = 0.2

fontsz=30

## Placement
scalebar_length_um = 15 # in microns
vertical_separation = 300
horizontal_separation = 50 # from left edge of movie to scalebar
text_bar_separation = 25
approx_length_of_text_px = 85
scalebar_height = 20
scalebar_bottom = x_size - vertical_separation
scalebar_top = scalebar_bottom - scalebar_height
scalebar_length_px = scalebar_length_um/scaling
scalebar_left_px = 0 + horizontal_separation              #scalebar_right_px - scalebar_length_px
scalebar_right_px = scalebar_left_px + scalebar_length_px #y_size-horizontal_separation
scalebar_left_percent = scalebar_left_px/(x_size*1.0)
#scalebar_length_percent = scalebar_length_px/(x_size*1.0)
#xmax = scalebar_length_percent + scalebar_left_percent
xmax = scalebar_right_px /(x_size*1.0)
center_of_bar = (scalebar_left_px + scalebar_right_px)/2.0
dpi = 72   # probably doesn't matter what this number is

frametime_sec = frametime/1000.0
moviegain=3

"""
Opens raw data frames from the disk, puts overlay over it,
writes new TIFF to disk. Memory semi-efficient :)

@param video_string - 'fueled' or 'control', tells function which TIFFs to open
@param name - filename of saved images
@param gain - multiply RGB values to better resolve particles from trajectories

@return None
"""
def make_movie_images_with_two_colors_new(video_string, name, gain=moviegain):
    green_particles, red_particles = (fueled_janus if video_string == 'fueled' else control_janus), (fueled_tracer if video_string == 'fueled' else control_tracer)
    green_tracks_to_show, red_tracks_to_show = green_particles['particle'].astype(int).unique(), red_particles['particle'].astype(int).unique()

   
    #length = len(os.listdir(movie_data_folder))
    frameno = 0
    
    for frame in sorted([item for item in np.sort(os.listdir(movie_data_folder)) if item.startswith(video_string)]):
        i = 0
        temp0 = tifffile.imread(os.path.join(movie_data_folder, frame))
        while i < len(temp0):
            #print(i)
            temp = temp0[i]
            #print(temp.shape)
            imagewidth=temp.shape[1]/dpi # calculate image width in inches
            imageheight=imagewidth*(temp.shape[0]/temp.shape[1])
            #print(imagewidth, imageheight)
            trajectorymovieframesize=[imagewidth,imageheight]

            fig = plt.figure(figsize=trajectorymovieframesize, frameon=False)
            #print(temp)
            ax = fig.add_axes([0,0,1,1])

            implot = ax.imshow(gain*temp)

            green_traj_hist = green_particles[green_particles.frame <= frameno]

            for this_particle in green_tracks_to_show:
                this_traj_hist = green_traj_hist[green_traj_hist.particle == this_particle]
                this_xcoords_hist = this_traj_hist['x']
                this_ycoords_hist = this_traj_hist['y']
                plot(this_xcoords_hist.values,this_ycoords_hist.values, 'g-', alpha=trajectory_transparency)

            #thisframefavcoords = green_particles[green_particles.frame == frameno]
            #xcoords, ycoords = thisframefavcoords['x'], thisframefavcoords['y']
            #plot(plt.xcoords.values, ycoords.values, 'r.', alpha=particle_transparency)
            
            red_traj_hist = red_particles[red_particles.frame <= frameno]
            
            for this_particle in red_tracks_to_show:
                this_traj_hist = red_traj_hist[red_traj_hist.particle == this_particle]
                this_xcoords_hist = this_traj_hist['x']
                this_ycoords_hist = this_traj_hist['y']
                plot(this_xcoords_hist.values,this_ycoords_hist.values, 'r-', alpha=trajectory_transparency)

            #thisframefavcoords = red_particles[green_particles.frame == frameno]
            #xcoords, ycoords = thisframefavcoords['x'], thisframefavcoords['y']
            #plot(plt.xcoords.values, ycoords.values, 'r.', alpha=particle_transparency)

            # scalebar
            plt.axhspan(scalebar_top, 
                    scalebar_top + scalebar_height, 
                    xmin=scalebar_left_percent, 
                    xmax=xmax,
                    color=scalebar_greyscale, 
                    alpha=scalebar_transparency)

            # scalebar label
            plt.text(center_of_bar-approx_length_of_text_px/2,scalebar_top-text_bar_separation, 
                 str(scalebar_length_um) + u' \u03bcm',
                 fontsize=fontsz,
                 color=scalebar_greyscale)

            # time stamp
            plt.text(horizontal_separation, horizontal_separation, "{:.1f}".format(frameno*frametime_sec) + ' sec',
                 fontsize=fontsz,
                 color=scalebar_greyscale)

            plt.ylim(y_size,0)
            plt.xlim(0,x_size)
            plt.xticks([],'')
            plt.yticks([],'')

            plt.savefig(os.path.join(movie_trajectory_folder, name + '_' + str(frameno).zfill(4) +'.tif'),
                   dpi = dpi)       

            temp0[i] = 0
            del temp
            i += 1
            frameno += 1
            
            close()
            
        
        del temp0
        gc.collect()

In [ ]:
""" 
Overlays only one group of particle trajectories over a TIFF and writes to disk.

@param particles - the pandas dataframe of particle trajectories
@param frames - the TIFF stack of particle motion
@param name - the name of the images being written on the disk
@param gain - multiply RGB values to better resolve particles from trajectories

@return None
"""
def make_movie_images_with_one_color(particles, frames, name, gain=1):
    tracks_to_show = particles['particle'].astype(int).unique()

    # Calculate image width and height in inches.
    imagewidth=frames.shape[2]/dpi # calculate image width in inches
    imageheight=imagewidth*(frames.shape[1]/frames.shape[2])
    trajectorymovieframesize=[imagewidth,imageheight]
    
    # Thanks to Becca Perry for helping me with this code.
    for i in range(len(frames)): 
        #print(i)
        fig = plt.figure(figsize=trajectorymovieframesize,frameon=False)
        ax = fig.add_axes([0,0,1,1])

        implot = ax.imshow(gain*frames[i])

        traj_hist =  particles[particles.frame <=i]

        for this_particle in tracks_to_show:
            this_traj_hist = traj_hist[traj_hist.particle == this_particle]
            this_xcoords_hist = this_traj_hist['x']
            this_ycoords_hist = this_traj_hist['y']
            plt.plot(this_xcoords_hist.values,this_ycoords_hist.values, 'r-', alpha=0.4)

        thisframefavcoords = particles[particles.frame ==i]
        plt.xcoords = thisframefavcoords['x']
        plt.ycoords = thisframefavcoords['y']
        plt.plot(plt.xcoords.values,plt.ycoords.values, 'r.', alpha=0.2)

        # scalebar
        plt.axhspan(scalebar_top, 
                scalebar_top + scalebar_height, 
                xmin=scalebar_left_percent, 
                xmax=xmax,
                color=scalebar_greyscale, 
                alpha=0.75)

        # scalebar label
        plt.text(center_of_bar-approx_length_of_text_px/2,scalebar_top-text_bar_separation, 
             str(scalebar_length_um) + u' \u03bcm',
             fontsize=fontsz,
             color=scalebar_greyscale)

        # time stamp
        plt.text(horizontal_separation, horizontal_separation, "{:.1f}".format(i*frametime_sec) + ' sec',
             fontsize=fontsz,
             color=scalebar_greyscale)

        plt.ylim(y_size,0)
        plt.xlim(0,x_size)
        plt.xticks([],'')
        plt.yticks([],'')

        plt.savefig(os.path.join(movie_trajectory_folder, name + '_' + str(i).zfill(4) +'.tif'),
               dpi = fig.dpi)

        if i < len(frames)-1:
            plt.close() # close all images except the last one.
        if i % 10 == 0:
            gc.collect()
    print('Saved images to:', os.path.join(movie_trajectory_folder, name + '_' + str(i).zfill(4) +'.tif'));

In [ ]:
if doing_control:
    make_movie_images_with_one_color(particles = control_tracer,  frames=pims.Video(control_file), 
                                     name=movie_name, gain=1)
    

In [ ]:
if doing_fueled:
    make_movie_images_with_two_colors_new(video_string = 'fueled', name = 'fueled')

In [ ]:
if doing_control:
    make_movie_images_with_two_colors_new(video_string = 'control', name = 'control')    

In [ ]:
import cv2

using_tif = False


"""
Open the recently written TIFFs with overlay and stitch them into an .avi file.

@param name - 'fueled' vs 'control'; which images to read.
@return None
"""
def make_movie_video(name):
    os.chdir(movie_trajectory_folder)
    if using_tif:
        print('Looking in folder:', movie_trajectory_folder)
        images = [img for img in np.sort(os.listdir(movie_trajectory_folder)) if (img.startswith(name) and img.endswith('.tif'))]
        frame = cv2.imread(os.path.join(movie_trajectory_folder, images[0]))

    else:
        images = pims.Video(control_file)
        frame = images[0]
    height, width, layers = frame.shape

    video = cv2.VideoWriter(name + ".avi", 0, 15, (width, height))
    print('Saved:', str(name) + '.avi')

    
    for image in images:
        video.write(cv2.imread(os.path.join(movie_trajectory_folder, image)))
        #os.remove(os.path.join(movie_trajectory_folder, image))
    

    cv2.destroyAllWindows()
    video.release()

#Might only work with the .tif
if doing_fueled:
    make_movie_video(fueled_movie_filename)
if doing_control:
    make_movie_video(movie_name + '_CONTROL')

os.chdir(home)

In [ ]:
gc.collect()